In [1]:
%%time
import src
from pathlib import Path
from tqdm.notebook import trange, tqdm
from src import Petri_Cheb_GNN, Petri_GCN

batch_size = 64
train_dataset = src.get_reachability_dataset(Path('Data/RandData_DS1_train_data.processed'), batch_size=batch_size)
test_dataset = src.get_reachability_dataset(Path('Data/RandData_DS1_test_data.processed'), batch_size=batch_size)

CPU times: user 2.28 s, sys: 1.01 s, total: 3.29 s
Wall time: 2.57 s


In [2]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, MLP
from torch_geometric.utils import scatter

class GCN(torch.nn.Module):
    def __init__(self, num_node_features, num_hidden_features):
        super().__init__()
        self.conv1 = GCNConv(num_node_features, num_hidden_features)
        self.conv2 = GCNConv(num_hidden_features, num_hidden_features)
        self.readout = MLP([num_hidden_features, num_hidden_features*2, num_hidden_features*3, 1])

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = self.readout(x)
        return scatter(x, data.batch, dim=0, reduce='mean').view(-1)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
def train():
    model.train()
    for epoch in tqdm(range(100), leave=False):
        for graph in train_dataset:
            optimizer.zero_grad()
            out = torch.flatten(model(graph))
            loss = F.l1_loss(out, graph.y)
            loss.backward()
            optimizer.step()

In [5]:
def test():
    model.eval()
    pred = []
    actual = torch.tensor([graph.y for graph in test_dataset.dataset])
    for graph in test_dataset:
        pred.extend(model(graph).tolist())
    
    pred = torch.tensor(pred)
    pred = torch.flatten(pred)

    mre = torch.mean(F.l1_loss(pred, actual, reduction='none') / actual) * 100
    return F.l1_loss(pred, actual), torch.nn.MSELoss()(pred, actual), mre

In [6]:
values = []
for i in trange(30):
    model = Petri_Cheb_GNN(train_dataset.num_features, 16, 3)
    # model = torch.compile(model, dynamic=True)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    train()
    values.append(test())
values = torch.tensor(values)

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [7]:
def variance(sample):
    return torch.mean((sample - torch.mean(sample))**2)

def std_deviation(sample):
    return variance(sample)**2

In [10]:
print(f"MAE Mean: {torch.mean(values[:, 0])}")
print(f"MSE Mean: {torch.mean(values[:, 1])}")
print(f"MRE Mean: {torch.mean(values[:, 2])}")

print(f"MAE Standard Deviation: {std_deviation(values[:, 0])}")
print(f"MSE Standard Deviation: {std_deviation(values[:, 1])}")
print(f"MRE Standard Deviation: {std_deviation(values[:, 2])}")

print(f"MAE Variance: {variance(values[:, 0])}")
print(f"MSE Variance: {variance(values[:, 1])}")
print(f"MRE Variance: {variance(values[:, 2])}")

MAE Mean: 0.21426576375961304
MSE Mean: 0.07458734512329102
MRE Mean: 6.119755268096924
MAE Standard Deviation: 2.563876364547468e-07
MSE Standard Deviation: 9.524162436491679e-08
MRE Standard Deviation: 0.18180827796459198
MAE Variance: 0.0005063473363406956
MSE Variance: 0.00030861241975799203
MRE Variance: 0.42638981342315674


In [9]:
values

tensor([[0.2026, 0.0641, 5.7791],
        [0.2148, 0.0747, 6.0652],
        [0.2259, 0.0803, 6.3969],
        [0.1912, 0.0568, 5.5027],
        [0.2000, 0.0624, 5.7379],
        [0.2210, 0.0788, 6.2402],
        [0.2533, 0.1008, 7.1511],
        [0.1924, 0.0595, 5.6395],
        [0.2467, 0.0967, 6.9713],
        [0.2136, 0.0727, 6.0559],
        [0.1945, 0.0589, 5.5627],
        [0.2019, 0.0641, 5.7964],
        [0.1949, 0.0604, 5.6934],
        [0.2270, 0.0813, 6.4302],
        [0.2127, 0.0729, 6.0328],
        [0.2277, 0.0870, 6.4218],
        [0.1909, 0.0591, 5.4714],
        [0.1987, 0.0623, 5.6809],
        [0.2069, 0.0720, 5.8743],
        [0.2904, 0.1426, 8.6574],
        [0.1834, 0.0505, 5.3636],
        [0.2185, 0.0779, 6.2053],
        [0.1944, 0.0699, 5.5733],
        [0.2229, 0.0805, 6.3175],
        [0.2459, 0.0954, 6.9390],
        [0.2292, 0.0828, 6.4881],
        [0.1940, 0.0596, 5.5655],
        [0.2150, 0.0746, 6.1138],
        [0.2135, 0.0726, 6.0458],
        [0.204